---
**License**

 TestInterp

 Mon Jan 25 20:56:00 2020\
 Copyright  2021\
 Eduardo ...\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 TestInterp\
 Copyright (C) 2021 Eduardo ...\
 Copyright (C) 2021 Sandro Dias Pinto Vitenti <vitenti@uel.br>


 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [ ]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import sys
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
from scipy.optimize import minimize

from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
%matplotlib inline

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [ ]:
    dim = 5
    np.random.seed(seed=123)

    p = np.random.random_sample((dim,))
    print (p)

In [ ]:
rng = Ncm.RNG.seeded_new (None, 123)

fmodel = Ncm.ModelMVND.new (dim)
fdata = Ncm.DataGaussCovMVND.new_full (dim, 0.1, 0.4, 10.0, -1.0, 1.0, rng)

fdata.props.use_norma = True

mset = Ncm.MSet.new_array ([fmodel])
mset.param_set_all_ftype (Ncm.ParamType.FREE)
mset.prepare_fparam_map ()

mset.fparams_set_array (p)
mset.pretty_log ()

In [ ]:
y_a = []
x_a = []
xn_a = []

interp = Ncm.StatsDistNdVBKStudentt.new (len (p), Ncm.StatsDistNdCV.NONE, 3.0)
#interp = Ncm.StatsDistNdKDEGauss.new (len (p), Ncm.StatsDistNdCV.NONE)
nps = 500

for a in range (nps):
    v, N = fdata.gen (mset, None, None, rng)
    interp.add_obs (v)
    y_a.append (fdata.m2lnL_val (mset))
    x_a.append (v.dup_array ())

for a in range (nps):
    v, N = fdata.gen (mset, None, None, rng)
    xn_a.append (v.dup_array ())

y_a = np.array (y_a)
x_a = np.array (x_a)
xn_a = np.array (xn_a)

fdata.y.set_array (p)

fdata.m2lnL_val (mset)

In [ ]:
#interp.set_cv_type (Ncm.StatsDistNdCV.SPLIT_FITD)
interp.set_cv_type (Ncm.StatsDistNdCV.NONE)
interp.prepare_interp (Ncm.Vector.new_array (y_a))
calib_os = interp.get_over_smooth ()

print (interp.get_rnorm (), calib_os)




In [ ]:
print (calib_os)

In [ ]:
#interp.set_over_smooth (calib_os)
#interp.prepare_interp (Ncm.Vector.new_array (y_a))

In [ ]:
       
yi_a = []
ya_a = []
yi_b = []

for x in xn_a:
    mset.fparams_set_array (x)
    fdata.prepare (mset)
    yi_a.append (interp.eval_m2lnp (Ncm.Vector.new_array (x)))
    ya_a.append (fdata.m2lnL_val (mset))

yi_a = np.array (yi_a)
ya_a = np.array (ya_a)

m = -np.mean (yi_a - ya_a)
diff = (yi_a + m) / ya_a - 1.0

probs = np.minimum(np.exp (-0.5*((yi_a[0::2]-ya_a[0::2])-(yi_a[1::2]-ya_a[1::2]))), 1.0)
bfact = np.abs(np.exp (-0.5*((yi_a[0::2]-ya_a[0::2])-(yi_a[1::2]-ya_a[1::2])))-1.0)

mean_err = 10**(np.mean (np.log10 (np.abs (diff))))

print ("mean error  %.2e" % (mean_err))
print ("Prob accept %.1f%%" % (100.0 * np.mean (probs)))

plt.figure (figsize=(14, 7))

plt.hist (probs, label='prob', bins = 60, density=True)
plt.grid ()
plt.legend(loc='best')
plt.show ()

plt.figure (figsize=(14, 7))

plt.hist (np.log10 (np.abs (diff)), bins = 60, label=r'$|y_i/y_a - 1|$')
plt.grid ()
plt.legend(loc='best')
plt.show ()

plt.figure (figsize=(14, 7))
plt.hist ((yi_a - ya_a), label='diff', bins = 60, density=True)